# Creating multi-agent collaboration: Slack Assistant
In this folder we will create the supervisor agent for our slack assistant and attach the grafana assistant and github assistant as sub-agents to it.

This agent will make usage of the `SUPERVISOR_ROUTER` collaboration mode in order to get the user's intent classification routed to the correct sub-agent

The agent architecture looks as following:
![Architecture](images/architecture.png)


## Prerequisites
Before starting, let's install the required boto3 libraries. Since this is a preview, the required libraries are being installed from the provided `whl` files

**Important:** this command will result in some conflict errors. It is ok to ignore those for now! We will check the pip versions in the next step

In [ ]:
!python3 -m pip install --force-reinstall --no-cache -q -r requirements.txt

## Import required libraries
Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:
- `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties
- `associate_sub_agents`: associate existing agents as sub-agents to the created supervisor agent
- `list_agent_collaborators`: list the available agents collaborations
- `invoke_agent_helper`: helps you to invoke your agent using invoke_agent

You can see the implementation of both functions in the parent directory

In [ ]:
import sys
import os
import time

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from agents import create_agent, associate_sub_agents, list_agent_collaborators, invoke_agent_helper

## Load Data From Stored Variables
Let's now load the sub-agents ARN from the stored variables and confirm that we've gotten the right values

In [ ]:
%store -r

In [ ]:
grafana_assistant_agent_alias_arn, github_assistant_agent_alias_arn

## Defining supervisor agent configuration

Let's now define the configuration for our travel assistant supervisor agent. Let's use the follow instructions to create our agent:

```
You are a supervisor agent responsible for coordinating between a GitHub agent and a Grafana agent. Your role is to:

1. Analyze user queries and determine which agent(s) should handle the request
2. Route requests to the appropriate agent(s)
3. Combine responses when needed
4. Ensure smooth interaction between agents when a task requires both GitHub and monitoring data

Guidelines for request handling:

1. For GitHub-related queries (involving repositories, pull requests, code, or GitHub operations):
   - Route to the GitHub agent
   - Keywords to watch for: "PR", "pull request", "repo", "repository", "GitHub", "commit"

2. For monitoring and alert-related queries:
   - Route to the Grafana agent
   - Keywords to watch for: "alert", "monitoring", "status", "metrics", "Grafana", "memory", "CPU"

3. For complex queries requiring both systems:
   - Break down the request into sub-tasks
   - Route each sub-task to the appropriate agent
   - Combine the responses in a meaningful way
   - Example: "Show me all PRs for apps with active alerts"

Response formatting:

1. Clearly indicate which agent provided which part of the response
2. Maintain context between related pieces of information
3. Present combined information in a logical and easy-to-understand format

Error handling:

1. If an agent cannot process a request, relay the error and suggest alternatives
2. If unsure about which agent should handle a request, ask the user for clarification
3. Ensure that partial failures don't prevent the delivery of available information

When interacting with users:
1. Maintain a helpful and professional tone
2. Clearly communicate which system is being queried
3. Ask for clarification when needed to route requests properly

Remember: Your primary role is to coordinate and ensure effective communication between the specialized agents while providing a seamless experience for the user.
```

For this agent, we will use `Claude 3.5 Sonnet v2` model in order to provide better reasoning capabilities to the supervisor agent.

As subagents, we have the following configuration:
* **GrafanaAssistant**: Agent that handles Grafana alerts and dimensional state analysis while strictly operating within accessed data and letting other agents handle their specific domains.
* **GithunAssistant**: Agent that handles GitHub operations like finding and searching pull requests while letting other agents handle their specific domains.

In [ ]:
agent_name = 'slack-poc-agent'
agent_foundation_model = "anthropic.claude-3-5-sonnet-20241022-v2:0"
agent_instruction = """You are a supervisor agent responsible for coordinating between a GitHub agent and a Grafana agent. Your role is to:

1. Analyze user queries and determine which agent(s) should handle the request
2. Route requests to the appropriate agent(s)
3. Combine responses when needed
4. Ensure smooth interaction between agents when a task requires both GitHub and monitoring data

Guidelines for request handling:

1. For GitHub-related queries (involving repositories, pull requests, code, or GitHub operations):
   - Route to the GitHub agent
   - Keywords to watch for: "PR", "pull request", "repo", "repository", "GitHub", "commit"

2. For monitoring and alert-related queries:
   - Route to the Grafana agent
   - Keywords to watch for: "alert", "monitoring", "status", "metrics", "Grafana", "memory", "CPU"

3. For complex queries requiring both systems:
   - Break down the request into sub-tasks
   - Route each sub-task to the appropriate agent
   - Combine the responses in a meaningful way
   - Example: "Show me all PRs for apps with active alerts"

Response formatting:

1. Clearly indicate which agent provided which part of the response
2. Maintain context between related pieces of information
3. Present combined information in a logical and easy-to-understand format

Error handling:

1. If an agent cannot process a request, relay the error and suggest alternatives
2. If unsure about which agent should handle a request, ask the user for clarification
3. Ensure that partial failures don't prevent the delivery of available information

When interacting with users:
1. Maintain a helpful and professional tone
2. Clearly communicate which system is being queried
3. Ask for clarification when needed to route requests properly

Remember: Your primary role is to coordinate and ensure effective communication between the specialized agents while providing a seamless experience for the user."""
agent_description = "Multi-agent collaboration for slack"

sub_agents_list = [
    {
        'sub_agent_alias_arn': grafana_assistant_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to handle Grafana alerts and dimensional state analysis while strictly operating within accessed data and letting other agents handle their specific domains.""",
        'sub_agent_association_name': 'GrafanaAssistant',
        'relay_conversation_history': 'DISABLED'
    },
    {
        'sub_agent_alias_arn': github_assistant_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to handle GitHub operations like finding and searching pull requests while letting other agents handle their specific domains.""",
        'sub_agent_association_name': 'GithubAssistant',
        'relay_conversation_history': 'DISABLED'
    }
]

## Create agent
Next we will create the agent with the provided information

In [ ]:
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn = create_agent(
    agent_name,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    agent_collaboration='SUPERVISOR_ROUTER',
    create_alias=False # working with test alias
)
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn

## Create agent associations
Next we will associate the required sub-agents to our supervisor agent

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn

## List agent associations
After that we can list the associations of our agent

In [ ]:
print(list_agent_collaborators(supervisor_agent_id))
time.sleep(10)

## Testing Agent
Now that we've created the agent, let's test it by using our `invoke_agent_helper` function

### Using Grafana Assistant
First let's invoke the slack assistant with a query that triggers the grafana assistant

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "can you get me alert history of memory alert for the app app1"

response = invoke(
    input_text=query, session_id=session_id, agent_id=supervisor_agent_id, 
    agent_alias_id=supervisor_agent_alias_id, enable_trace=True
)

print(response)

### Using Github Assistant
Next let's invoke our slack assistant with a query that triggers the github assistant. Before doing that we will sleep for 60 seconds to avoid throttling. This can be removed for accounts with higher limits

In [ ]:
time.sleep(60)

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "get me all github related pull requests for owner mttanke for repo amazon-bedrock-samples"

response = invoke(
    input_text=query, session_id=session_id, agent_id=supervisor_agent_id, 
    agent_alias_id=supervisor_agent_alias_id, enable_trace=True
)

print(response)

## Saving the information
Let's now save some information so that we can use this agent as a sub-agent of our traveler assistant

In [ ]:
supervisor_agent_id, supervisor_agent_alias_id

In [ ]:
%store supervisor_agent_id
%store supervisor_agent_alias_id

In [ ]:
supervisor_agent_id

## Next steps

So far we have created a supervisor agent and invoked it with the buildtime boto3 SDK. This allows us to test the agent behavior but does not provide the latest functionalities for the tracing. Let's now test those functionalities using the runtime boto3 SDK